In [1]:
import selenium
from selenium import webdriver
# 下载后还要去下载一个对应的浏览器的driver，并且把exe程序放在你的python或者anaconda
# 的 Scripts目录下
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from lxml import etree
import pandas as pd

#实例化一个浏览器
driver = webdriver.Chrome()
# 发送请求
url = 'https://www.baidu.com/'
driver.get(url)

In [14]:
def open51job(url,keys):
    driver = webdriver.Chrome()
    driver.get(url)
    print('进入:',driver.title)
    #进入页面，找到搜索框(定位)
    elem = driver.find_element_by_id('kwdselectid')
    elem.clear()  #输入框如果有原始数据就删除掉
    elem.send_keys(keys)
    sleep(1)
    #设置城市
    elem.find_element_by_xpath('//*[@id="work_position_input"]').click() #点击工作输入
    sleep(1)
    elem.find_element_by_xpath('//*[@id="work_position_click_multiple_selected_each_360000"]/em').click() #点掉国外
    sleep(1)
    
    elem.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_040000"]').click()
    
#     elem.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_040000"]').click()# 点击深圳地区
    elem.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_010000"]').click()
    elem.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_020000"]').click()
    elem.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_030200"]').click()
    elem.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_080200"]').click()
#//*[@id="work_position_click_center_right_list_category_000000_010000"]北京
#//*[@id="work_position_click_center_right_list_category_000000_020000"]上海
#//*[@id="work_position_click_center_right_list_category_000000_030200"]广州
#//*[@id="work_position_click_center_right_list_category_000000_080200"]
    sleep(1)
    elem.find_element_by_xpath('//*[@id="work_position_click_bottom_save"]').click()  #点击确定
    
    sleep(1)
    elem.find_element_by_xpath('/html/body/div[3]/div/div[1]/div/button').click()
    
    #//*[@id="work_position_click"]
    sleep(1)
    
    #进入了新页面后
    search_widow = driver.current_window_handle
    
    driver.find_elements_by_xpath('//*[@id="filter_providesalary"]/span[3]')[0].click()
    driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[8]/a/em').click()
    driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[9]/a/em').click()
    driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[10]/a/em').click()
    driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[11]/a/em').click()
    driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[12]/a/em').click()
    driver.find_element_by_xpath('//*[@id="submit_providesalary"]').click()
    
    
    # 获取所有窗口
    # for handls in driver.window_handles:
    #    drivet.switch_to_window(handle) 切换窗口
    return driver




    

In [15]:

##############注意，必须放在函数外。因为没执行一次就想当于更新了infos变成空
#infotemp={}
infos=[]
def searchJob(driver): #拿到当前浏览器的对象
    
    page_source = driver.page_source # 网页源代码
    
    # 匹配具体职位信息

    html = etree.HTML(page_source)
    divs = html.xpath('//div[@id=\'resultList\']/div[@class=\'el\']')
    
    i=1
    for div in divs:
        ####################  取得职位信息，如果为空，显示no,display!###############################
        occupation = div.xpath('.//p/span/a/@title')
        occupation.append('No Display')
        occupation = occupation[0]
        ###########################################################
        #infotemp['职位名']=occupation
        ###################################
        
        href_occupation = div.xpath('.//p/span/a/@href')
        href_occupation.append('No Display')
        href_occupation = href_occupation[0]
        #infotemp['职位链接']=href_occupation
        
        company = div.xpath('.//span[@class="t2"]/a/@title')
        company.append('No Display')
        company = company[0]
        #infotemp['公司名']=company
        
        address = div.xpath('.//span[@class="t3"]/text()')
        address.append('No Display')
        address = address[0]
        #infotemp['公司地点'] = address
        
        salary = div.xpath('.//span[@class="t4"]/text()')
        salary.append('No Display')
        salary = salary[0]
        #infotemp['薪资']=salary

        publish = div.xpath('.//span[@class="t5"]/text()')
        publish.append('No Display')
        publish = publish[0]  #  把string类型提取出来，因为一般默认为列表
        #infotemp['发布时间']=publish
        
        ######最后加到列表里#########
        #print(infotemp)
        
        ##不要用中间引用！
        infos.append({'职位名':occupation,'职位链接':href_occupation,'公司名':company,'公司地点':address,'薪资':salary,'发布时间':publish})
        infos_dataf = pd.DataFrame(infos,columns=['职位名','职位链接','公司名','公司地点','薪资','发布时间'])
        #print(infos_dataf)
    
        print('##################第'+str(i)+'个job############')
        print('职位名:'+occupation )
        print('职位链接:'+href_occupation)
        print('公司名:'+company)
        print('工作地点:'+address)
        print('薪资:'+salary)
        print('发布时间:'+publish)
        print('\n')
        i+=1
    return driver
        


In [16]:
def nextPage(driver):
    try:
        page_num = driver.find_element_by_link_text('下一页')
        page_num.click()
        
    except NoSuchElementException:
        print('搜索完毕')
        flag=0
        return flag

In [17]:
if __name__ == '__main__':
    url = 'http://www.51job.com'
    keys =input("请输入搜索的关键字：")
    num= 1
    driver = open51job(url,keys)
    while True:
        #print('####')
        driver =searchJob(driver)
        flag = nextPage(driver)
        if flag==0:
            break
        num+=1
    driver.close()
    infos_df = pd.DataFrame(infos,columns=['职位名','职位链接','公司名','公司地点','薪资','发布时间'])
    path = input('请你输入要保存职位信息的path：')
    name_file = input('请输入excel的文件名:')
    infos_df.to_excel(path+'\\'+name_file+'.xlsx',encoding='utf-8',sheet_name='hardware')
    driver.quit()

请输入搜索的关键字：物理治疗
进入: 招聘网_人才网_找工作_求职_上前程无忧
##################第1个job############
职位名:物理治疗师
职位链接:https://jobs.51job.com/guangzhou-yxq/111572246.html?s=01&t=0
公司名:广州启时贸易有限公司
工作地点:广州-越秀区
薪资:1-1.5万/月
发布时间:08-15


##################第2个job############
职位名:物理治疗师Physical therapist
职位链接:https://jobs.51job.com/shenzhen-lhq/114592619.html?s=01&t=0
公司名:深圳市前海宏景信息服务有限公司
工作地点:深圳-罗湖区
薪资:2-3万/月
发布时间:08-14


##################第3个job############
职位名:物理治疗师/康复治疗师
职位链接:https://jobs.51job.com/beijing-dcq/83068105.html?s=01&t=0
公司名:深圳市好家庭实业有限公司
工作地点:北京-东城区
薪资:1-1.5万/月
发布时间:08-12


##################第4个job############
职位名:物理治疗师
职位链接:https://jobs.51job.com/shanghai/108266125.html?s=01&t=0
公司名:北京欧培德科技有限公司
工作地点:上海
薪资:1-1.5万/月
发布时间:08-06


##################第5个job############
职位名:Senior PT高级物理治疗师
职位链接:https://jobs.51job.com/beijing-xcq/86836085.html?s=01&t=0
公司名:北京中康美复医疗管理咨询有限公司
工作地点:北京-西城区
薪资:1.5-2万/月
发布时间:07-08


##################第6个job############
职位名:康复医师
职位链接:https://jobs.51job.com/shanghai-pdxq/116043712.html?s

C:\Users\richa\Desktop\position_finding

pd.DataFrame?


import copy# 注意，不能把colLumns放在asdf之前！！
b=[]
a={'a':"c",'b':'c'}
ap =copy.deepcopy(a)
b.append(a)
b


ap['a']='c'
ap['b']='d'
b.append(a)
b



# 试验品：

url = 'http://www.51job.com'
keys =input("请输入搜索的关键字：")
num= 1
driver = webdriver.Chrome()
driver.get(url)
print('进入:',driver.title)
    #进入页面，找到搜索框(定位)
elem = driver.find_element_by_id('kwdselectid')
elem.clear()  #输入框如果有原始数据就删除掉
elem.send_keys(keys)
sleep(1)
    #设置城市
driver.find_element_by_xpath('//*[@id="work_position_input"]').click() #点击工作输入
sleep(1)
driver.find_element_by_xpath('//*[@id="work_position_click_multiple_selected_each_360000"]/em').click() #点掉国外
sleep(1)
#elem.find_element_by_xpath('//*[@id="work_position_click_ip_location_000000_360000"]').click()
#sleep(1)

driver.find_element_by_xpath('//*[@id="work_position_click_center_right_list_category_000000_040000"]').click()# 点击深圳地区
sleep(1)
driver.find_element_by_xpath('//*[@id="work_position_click_bottom_save"]').click()  #点击确定
    
sleep(1)
driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/div/button').click()
sleep(1)
    
    #进入了新页面后

sreach_window=driver.current_window_handle

#elem.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[8]/a/em').click()
#driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[9]/a/em').click()
#driver.find_element_by_xpath('//[@id="multichoices_providesalary"]/ul/li[10]/a/em').click()
#driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[11]/a/em').click()
#driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[12]/a/em').click()
#driver.find_element_by_xpath('//*[@id="submit_providesalary"]').click()
driver.find_element_by_xpath('//*[@id="filter_providesalary"]/span[3]').click()
driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[8]/a').click()
driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[9]/a').click()
driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[10]/a').click()
driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[11]/a').click()
driver.find_element_by_xpath('//*[@id="multichoices_providesalary"]/ul/li[12]/a').click()
driver.find_element_by_xpath('//*[@id="submit_providesalary"]').click()


# 试验
#encoding=utf-8